In [1]:
import json
import os
import numpy
import pandas
import pyarrow
import sys

from datetime import date
from os import path
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Steps to install
# 1. pip install sqlalchemy-bigquery google-cloud-bigquery-storage pyarrow
# 2. Copy the credentials file to wherever you set BIGQUERY_CREDENTIALS_PATH to

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')

In [2]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
import create_queries
%load_ext autoreload
%autoreload 2

In [3]:
COMPANY_NAME = 'Royal Apothecary'
COMPANY_IDENTIFIER = 'RA'
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'
INVENTORY_DATES = [
    '09/30/2020',
    '10/31/2020',
    '11/30/2020',
    '12/31/2020',
    '01/31/2021',
    '02/28/2021',
    '03/31/2021',
    '04/30/2021',
    '05/31/2021',
    '06/30/2021',
    '07/31/2021',
    '08/31/2021',
    '09/30/2021',
]
ANALYSIS_PARAMS = {
    'sold_threshold': 1.0
}
TODAY_DATE = date.today().strftime('%m/%d/%Y')
INVENTORY_DATES.append(TODAY_DATE)
print('Today is {}'.format(TODAY_DATE))

Today is 10/26/2021


In [4]:
# Download packages, sales transactions, incoming / outgoing tranfers

company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_outgoing_transfer_packages_query = create_queries.create_company_outgoing_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_sales_transactions_query = create_queries.create_company_sales_transactions_query(COMPANY_IDENTIFIER, SALES_TRANSACTIONS_START_DATE)
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(COMPANY_IDENTIFIER)

engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))
company_incoming_transfer_packages_dataframe = pandas.read_sql_query(company_incoming_transfer_packages_query, engine)
company_outgoing_transfer_packages_dataframe = pandas.read_sql_query(company_outgoing_transfer_packages_query, engine)
company_sales_transactions_dataframe = pandas.read_sql_query(company_sales_transactions_query, engine)
company_inventory_packages_dataframe = pandas.read_sql_query(company_inventory_packages_query, engine)

In [5]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts/analysis")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

from util import active_inventory_util as util

In [6]:
d = util.Download()
d.download_dataframes(
    incoming_transfer_packages_dataframe=company_incoming_transfer_packages_dataframe,
    outgoing_transfer_packages_dataframe=company_outgoing_transfer_packages_dataframe,
    sales_transactions_dataframe=company_sales_transactions_dataframe,
)

In [15]:
# Check for how many packages are excluded from analysis,
# seeing less then 2% is really good
q = util.Query()
q.inventory_dates = INVENTORY_DATES
q.company_name = COMPANY_NAME

id_to_history = util.get_histories(d)
util.print_counts(id_to_history)
util.create_inventory_xlsx(id_to_history, q, params=ANALYSIS_PARAMS)

Only outgoing: 7
Only incoming: 1531
Only sold: 11
In and out: 37
In and sold at least once 4123
In and sold many times 3995
Total pkgs: 5687
Wrote result to out/Royal Apothecary_inventory_by_month.xls
Excluded 18 / 5687 packages from consideration (0.32%)
  MISSING_INCOMING: 18 times


In [18]:
import importlib
importlib.reload(util)

# TODO(dlluncor): Just for debugging to make this faster
INVENTORY_DATES = [TODAY_DATE]

date_to_inventory_packages_dataframe = {}
id_to_history = util.get_histories(d)

for inventory_date in INVENTORY_DATES:
    computed_inventory_package_records = util.create_inventory_dataframe_by_date(
        id_to_history, inventory_date, params=ANALYSIS_PARAMS)    
    computed_inventory_packages_dataframe = pandas.DataFrame(
        computed_inventory_package_records,
        columns=[
            'package_id',
            'license_number',
            'arrived_date',
            'product_category_name',
            'product_name',
            'quantity',
            'sold_date',
            'is_in_inventory'
        ]
    )
    date_to_inventory_packages_dataframe[inventory_date] = computed_inventory_packages_dataframe

In [19]:
from_packages_inventory_dataframe = company_inventory_packages_dataframe[[
    'package_id',
    'packaged_date',
    'product_category_name',
    'product_name',
    'quantity',
]].sort_values('package_id')

In [54]:
# For debugging individual package histories
import importlib
importlib.reload(util)

# Missing transactions?
# 5189649

# Using grams or missing transactions?

PACKAGE_IDS = [
    '6880749'
]

util.analyze_specific_package_histories(d, PACKAGE_IDS, params=ANALYSIS_PARAMS)


Package 6880749 arrived on 08/26/2020 with quantity 132 and price $0.05. {'package_row_id': '8e2c7d19-13d3-497d-8be1-6a341b22a083', 'delivery_type': 'INCOMING_FROM_VENDOR', 'license_number': 'C10-0000596-LIC', 'manifest_number': '0000932239', 'created_date': datetime.date(2020, 8, 26), 'received_datetime': Timestamp('2020-08-26 23:47:49+0000', tz='UTC'), 'shipper_facility_license_number': 'C11-0000578-LIC', 'shipper_facility_name': '9701 ENTERPRISES, INC.', 'recipient_facility_license_number': 'C10-0000596-LIC', 'recipient_facility_name': 'ROYAL APOTHECARY L.L.C.', 'shipment_type_name': 'Wholesale Manifest', 'shipment_transaction_type': 'Wholesale', 'package_id': '6880749', 'package_label': '1A4060300006023000042353', 'type': 'transfer_outgoing', 'shipment_package_state': 'Accepted', 'is_testing_sample': False, 'is_trade_sample': False, 'product_category_name': 'Pre-Roll Flower', 'product_name': 'Sticks Preroll - Chocolate Fire - 1G / Case of 44', 'package_lab_results_status': 'passed

In [40]:
import importlib
importlib.reload(util)

res = util.compare_inventory_dataframes(
    computed=date_to_inventory_packages_dataframe[TODAY_DATE],
    actual=from_packages_inventory_dataframe
)

# Two categories for: Num actual packages not computed: 229
# 1. Actual has a package that we've never seen
# 2. Actual has a package that we've sold out of, and that's why it's in the inventory

# Two categories for: Num computed packages not in actual: 237
# 1. We never saw the package in the actual inventory
# 2. Actual inventory is sold out, but we think it's not sold out in computed

Pct of # inventory matching: 93.68%
Accuracy of quantities: 98.08%
Pct of # inventory packages over-estimated: 6.68%
Pct of # quantity over-estimated: 0.14%
Avg quantity delta: 0.11
Avg quantity: 5.88

Num matching packages: 3337
Num actual packages not computed: 225
  but computed at some point: 202, e.g., 89.78% of non-computed packages
  avg quantity from actual packages 1.30
Num computed packages not in actual: 238
  but in actual inventory at some point: 0

Computed has these extra package IDs; first 40
8973206; computed quantity 638
6880749; computed quantity 132
9195587; computed quantity 100
3576925; computed quantity 97
12664646; computed quantity 92
3578093; computed quantity 84
11763124; computed quantity 66
17562729; computed quantity 64
9821427; computed quantity 63
5033931; computed quantity 55
11763119; computed quantity 53
11677191; computed quantity 43
17481959; computed quantity 40
5262621; computed quantity 38
3576931; computed quantity 38
10190898; computed quantity

In [51]:
inactive_packages_df = pandas.read_sql_query(
    util.are_packages_inactive_query(res['computed_extra_package_ids']),
    engine
)
inactive_packages_df

,identifier,type,package_id,package_label,quantity


In [39]:
# Find transfer packages in data warehouse by package_id.

def create_transfer_packages_by_package_id_query(package_id):
    return f"""
        select
            companies.identifier,
            company_deliveries.delivery_type,
            company_deliveries.updated_at,
            metrc_transfer_packages.package_id,
            metrc_transfer_packages.package_label,
            metrc_transfer_packages.shipped_quantity,
            metrc_transfer_packages.shipper_wholesale_price,
            metrc_transfers.shipper_facility_name,
            metrc_transfers.shipper_facility_license_number,
            metrc_deliveries.recipient_facility_name,
            metrc_deliveries.recipient_facility_license_number
        from
            metrc_transfer_packages
            inner join metrc_deliveries on metrc_transfer_packages.delivery_row_id = metrc_deliveries.id
            inner join metrc_transfers on metrc_deliveries.transfer_row_id = metrc_transfers.id
            inner join company_deliveries on metrc_transfers.id = company_deliveries.transfer_row_id
            inner join companies on company_deliveries.company_id = companies.id
        where
            True
            and metrc_transfer_packages.package_id = '{package_id}'
    """

transfer_packages_by_package_id_query = create_transfer_packages_by_package_id_query(
    8973206)
transfer_packages_by_package_id_dataframe = pandas.read_sql_query(transfer_packages_by_package_id_query, engine)
transfer_packages_by_package_id_dataframe

,identifier,delivery_type,updated_at,package_id,package_label,shipped_quantity,shipper_wholesale_price,shipper_facility_name,shipper_facility_license_number,recipient_facility_name,recipient_facility_license_number
0,None,OUTGOING_TO_PAYOR,2021-09-19 19:55:04.741000+00:00,8973206,1A40603000042E6000000001,1573.0,None,THE HEALING HEART COLLECTIVE,C11-0000059-LIC,ROYAL APOTHECARY L.L.C.,C10-0000596-LIC
1,RA,INCOMING_FROM_VENDOR,2021-09-19 19:55:04.822000+00:00,8973206,1A40603000042E6000000001,1573.0,None,THE HEALING HEART COLLECTIVE,C11-0000059-LIC,ROYAL APOTHECARY L.L.C.,C10-0000596-LIC


In [18]:
# Find packages in data warehouse by package_id.

def create_packages_by_package_id_query(package_id):
    return f"""
        select
            companies.identifier,
            metrc_packages.package_id,
            metrc_packages.package_label,
            metrc_packages.quantity
        from
            metrc_packages
            inner join companies on metrc_packages.company_id = companies.id
        where
            True
            and metrc_packages.package_id = '{package_id}'
    """

packages_by_package_id_query = create_packages_by_package_id_query(17481729)
packages_by_package_id_dataframe = pandas.read_sql_query(packages_by_package_id_query, engine)
packages_by_package_id_dataframe

,identifier,package_id,package_label,quantity
0,RA,17481729,1A4060300003BC9000058384,1.0



Package 5751573 arrived on 07/13/2020 with quantity 280 and price $350.0. {'package_row_id': '4544aa24-1a3d-494b-b312-de17db773e79', 'delivery_type': 'INCOMING_FROM_VENDOR', 'license_number': 'C10-0000596-LIC', 'manifest_number': '0000767939', 'created_date': datetime.date(2020, 7, 13), 'received_datetime': Timestamp('2020-07-15 00:56:06+0000', tz='UTC'), 'shipper_facility_license_number': 'C11-0000349-LIC', 'shipper_facility_name': 'MISSION HILLS PATIENTS COLLECTIVE', 'recipient_facility_license_number': 'C10-0000596-LIC', 'recipient_facility_name': 'ROYAL APOTHECARY L.L.C.', 'shipment_type_name': 'Wholesale Manifest', 'shipment_transaction_type': 'Wholesale', 'package_id': '5751573', 'package_label': '1A4060300003D59000009129', 'type': 'transfer_incoming', 'shipment_package_state': 'Accepted', 'is_testing_sample': False, 'is_trade_sample': False, 'product_category_name': 'Flower', 'product_name': 'Rich & Ruthless - CPT Kush Ride (28 GM Flower - SMS) (#112-CPTK-YF0420) #1884', 'packa